### loading raw json file into dataframe df

In [3]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c06a5671-092d-483a-b7c0-4f8d42a0a4de)

### extracting actual values we need

In [4]:
df = df.select("value")

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 6, Finished, Available, Finished)

In [5]:
display(df)

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f2cd4c6c-89bb-4e8b-9561-6e616bf7ca23)

### change from single row to multiple using explode function

In [35]:
from pyspark.sql.functions import explode
df_exploded = df.select(explode(df["value"].alias("col")))

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 37, Finished, Available, Finished)

In [36]:
display(df_exploded)

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 38, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 659701f3-e1d0-4993-bb51-181eaa91214b)

In [37]:
json_list = df_exploded.toJSON().collect()

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 39, Finished, Available, Finished)

### checking the json_list

In [38]:
print(json_list)

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 40, Finished, Available, Finished)

['{"col":{"about":[{"name":"Liverpool","readLink":"https://api.bing.microsoft.com/api/v7/entities/a5642e81-20ab-a561-17cc-52a63926b210"},{"name":"Teun Koopmeiners","readLink":"https://api.bing.microsoft.com/api/v7/entities/ece7389b-2994-2be9-fa34-e03b0b5e026e"},{"name":"Mohamed Salah","readLink":"https://api.bing.microsoft.com/api/v7/entities/fbd8133b-7464-418f-a557-67bfb2281bbd"},{"name":"Juventus F.C.","readLink":"https://api.bing.microsoft.com/api/v7/entities/629fec02-ac1f-5a01-04a6-ba9246703d7b"},{"name":"Serie A","readLink":"https://api.bing.microsoft.com/api/v7/entities/bd48a5b1-961c-df36-94e0-ff284105d179"},{"name":"A.C. Monza","readLink":"https://api.bing.microsoft.com/api/v7/entities/fe742275-3d0c-c1f1-d7c4-cef4f869623a"},{"name":"Turin","readLink":"https://api.bing.microsoft.com/api/v7/entities/cd6a2d4d-bc1e-3b45-c388-9f29744c55ea"}],"category":"Sports","datePublished":"2024-07-06T18:43:00.0000000Z","description":"Latest Liverpool transfer news and gossip including updates on

### checking value at 0

In [39]:
print(json_list[0])

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 41, Finished, Available, Finished)

{"col":{"about":[{"name":"Liverpool","readLink":"https://api.bing.microsoft.com/api/v7/entities/a5642e81-20ab-a561-17cc-52a63926b210"},{"name":"Teun Koopmeiners","readLink":"https://api.bing.microsoft.com/api/v7/entities/ece7389b-2994-2be9-fa34-e03b0b5e026e"},{"name":"Mohamed Salah","readLink":"https://api.bing.microsoft.com/api/v7/entities/fbd8133b-7464-418f-a557-67bfb2281bbd"},{"name":"Juventus F.C.","readLink":"https://api.bing.microsoft.com/api/v7/entities/629fec02-ac1f-5a01-04a6-ba9246703d7b"},{"name":"Serie A","readLink":"https://api.bing.microsoft.com/api/v7/entities/bd48a5b1-961c-df36-94e0-ff284105d179"},{"name":"A.C. Monza","readLink":"https://api.bing.microsoft.com/api/v7/entities/fe742275-3d0c-c1f1-d7c4-cef4f869623a"},{"name":"Turin","readLink":"https://api.bing.microsoft.com/api/v7/entities/cd6a2d4d-bc1e-3b45-c388-9f29744c55ea"}],"category":"Sports","datePublished":"2024-07-06T18:43:00.0000000Z","description":"Latest Liverpool transfer news and gossip including updates on T

### trnasforming data from string to dictionary

In [40]:
import json

news_json = json.loads(json_list[0])

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 42, Finished, Available, Finished)

In [41]:
print(news_json['col']['name'])
print(news_json['col']['description'])
print(news_json['col']['category'])
print(news_json['col']['url'])
print(news_json['col']['image']['thumbnail']['contentUrl'])
print(news_json['col']['provider'][0]['name'])
print(news_json['col']['datePublished'])


StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 43, Finished, Available, Finished)

Liverpool transfer news LIVE after Trent Alexander-Arnold heroics, Ibrahima Konate blunt response and Nico Williams U-turn
Latest Liverpool transfer news and gossip including updates on Teun Koopmeiners, Mohamed Salah, Michele Di Gregorio and more
Sports
https://www.liverpoolecho.co.uk/sport/football/transfer-news/liverpool-transfer-news-live-koopmeiners-29487319
https://www.bing.com/th?id=OVFT.WOB71ve6f0zghJoaO7mMoS&pid=News
Liverpool Echo
2024-07-06T18:43:00.0000000Z


title,
description,
category,
url,
image,
provider,
datePublished

### transforming all the data using for loop, if image and category is missing data is ignored

In [42]:
title = []
description =[]
category =[]
url =[]
image =[]
provider =[]
datePublished =[]

#process each JSON object in the list
for json_str in json_list:
    try:
        #parse the JSON string into dictionary
        article = json.loads(json_str)

        if article["col"].get("category") and article["col"].get("image",{}).get("thumbnail",{}).get("contentUrl"):
            #extract information from the dictionary
            title.append(article["col"]["name"])
            description.append(article["col"]["description"])
            category.append(article["col"]["category"])
            url.append(article["col"]["url"])
            image.append(article["col"]["image"]["thumbnail"]["contentUrl"])
            provider.append(article["col"]["provider"][0]["name"])
            datePublished.append(article["col"]["datePublished"])

        

    except Exception as e:
        print(f"Error processing JSON object: {e}")

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 44, Finished, Available, Finished)

In [43]:
title

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 45, Finished, Available, Finished)

['Liverpool transfer news LIVE after Trent Alexander-Arnold heroics, Ibrahima Konate blunt response and Nico Williams U-turn',
 "Arsenal transfer news LIVE - Raphinha claim amid Pedro Neto 'approach' and Saliba interest",
 "Sky News voters panel optimistic about new Labour government - but here's why some are worried",
 'Arsenal transfer news LIVE: Gunners ‘agree contract terms for Calafiori’, £27m signing CONFIRMED, Williams latest',
 'Rangers transfer news latest on Dessers, Danilo and new striker plan as Dapo Mebude in line for emotional SPFL return',
 'Man Utd transfer news LIVE: Chiesa BOOST, French giants ‘want Greenwood’, United prepared to SELL Rashford – updates',
 "Transfer news LIVE: Arsenal's Calafiori bid accepted, Chelsea's Williams advantage, Man Utd latest",
 'Mumbai News Live Updates: Ruling Mahayuti alliance to meet at 5 pm today; CM Shinde to address gathering',
 'Epoch Times, the conspiratorial pro-Trump outlet, enters a new market: faith-based movies',
 'Man City 1

### coverting the list to dataframe

In [44]:
from pyspark.sql.types import StructType, StructField, StringType

#combine the lists
data = list(zip(title, description, category, url, image, provider, datePublished))

#define schema
schema = StructType([
    StructField("title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("url", StringType(), True),
    StructField("image", StringType(), True),
    StructField("provider", StringType(), True),
    StructField("datePublished", StringType(), True)
])
df_cleaned = spark.createDataFrame(data, schema= schema)

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 46, Finished, Available, Finished)

In [45]:
display(df_cleaned)

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 47, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 9f6a4312-9be8-401f-a585-345fc53e4ecc)

### trnasforming "datePublished" column

In [46]:
from pyspark.sql.functions import to_date, date_format
df_cleaned_final = df_cleaned.withColumn("datePublished", date_format(to_date("datePublished"), "dd-mm-yyyy"))

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 48, Finished, Available, Finished)

In [47]:
display(df_cleaned_final)

StatementMeta(, 2cbbc2e6-37b8-4201-94d2-d8ef0d66ea18, 49, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7ccfe204-0a6a-4166-9738-51ca38bea35e)